## Import Libraries

In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate
from scipy.spatial import distance
import warnings
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [2]:
case  = 'wat'


if case=='atm':
    theta  = 0
    n      = 1.00029
    c      = 299792458/n
    h      = 25000
    L      = 100
    v      = n * c
    times  = linspace(1e-6,1e-4,100000,endpoint=True)
    
elif case=='wat':
    n      = 1.33
    c      = 299792458/n
    h      = 4.5
    D      = 7.3   #--------------------- diameter of tank
    R      = 7.3/2 #--------------------- radius of tank
    v      = n * c
    times  = linspace(1e-10,1e-7,200000)
    
    c1     = (0, 0, 0)  #------------------------------------ Central    PMT number 1 
    c2     = (1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0) #------ Non-Radial PMT number 2
    c3     = (1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0) #------ Non-Radial PMT number 3
    c4     = (1.85*cos(0     ) , 1.85*sin(0     ), 0) #------ Radial     PMT number 4
    x1,y1  = (1.5,1.5) #----------------------------------------- Coordinates of particle at top tanker lid      #------PARAM
    theta  = 180   #----------------------------------------- Zenith  varies from 90 to 180                    #------PARAM
    phi    = 0    #----------------------------------------- Azimuth varies from  0 to 360                    #------PARAM
    theta  = deg2rad(theta) #-------------------------------- Convert zenith to radians
    phi    = deg2rad(phi  ) #-------------------------------- Convert azimuth to radians

    x2     = x1 + h * tan(theta) * cos(phi) # ------------------- Calculate bottom x coordinate
    y2     = y1 + h * tan(theta) * sin(phi) #-------------------- Calculate bottom y coordinate
    cp     = (x2,y2,0)
    print('A = {}'.format((x1,y1,h)))
    print('B = {}'.format(cp))

    d1     = distance.euclidean(cp,c1)  #------------------------ Calculate the distances of the point 
    d2     = distance.euclidean(cp,c2)  #------------------------ where the particle hits the bottom of 
    d3     = distance.euclidean(cp,c3)  #------------------------ the tank from the 4 PMTs
    d4     = distance.euclidean(cp,c4)
    theta2    = pi - theta
    del theta
    theta     = theta2
    print('Particle distances from PMTs    : {} m,   {} m,   {} m,   {} m'.format( round(d1,3),round(d2,3),round(d3,3),round(d4,3)))

A = (1.5, 1.5, 4.5)
B = (1.4999999999999996, 1.5, 0)
Particle distances from PMTs    : 2.121 m,   2.427 m,   3.938 m,   1.54 m


## Define Entry Brightness 

In [3]:
def entry_brightness(times,h,c,v,theta,L):
    z        = h/cos(theta)
    k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
    t_shower = times[0]
    t_light  = k/c
    t_total  = t_shower + t_light
    tt       = t_total
    den      = (c**2-v**2)
    num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
    v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
    v_B      = num2_z
    vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
    vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
    kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
    km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
    alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
    alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
    vtp      = vp * sin(alphap)
    vtm      = vm * sin(alpham)
    omegap   = (vtp / kp)
    omegam   = (vtm / km)
    bp       = abs(omegap / (kp**2))
    bm       = abs(omegam / (km**2))
    return bp

In [ ]:
%matplotlib
#plt.figure(figsize=(12,9))

plotme = 'b vs phi'
if 'b' in plotme:
    ptype = str(input("Do you want normalized brightness? (Enter Y/y or N/n)"))
    print()
distances = [d1,d2]
colors    = ['r','k']
detectors = [1,2]

for ds, color, detector in zip(distances, colors, detectors):
    print("==============================================")
    print("Processing detector %s"%detector)
    print("==============================================")
    
    T, ZP, ZM, BP, BM, PHIP, PHIM = [], [], [], [], [], [], []
    
    #if ds<1e-12:
    #    L=0.0
    #else:
    L      = ds
    
    num_zc = -sqrt(-c**4 * L**2 * cos(theta)**2 + c**2 * L**2 * v**2 * cos(theta)**2) + c**2*L*sin(theta) - L*v**2*sin(theta)
    den    = (c**2-v**2)
    zc     = (num_zc/den)
    zc_act = zc * cos(theta)
    print('First RID Height: {} m'.format(zc_act))

    
    for iii,t in enumerate(times):
        z        = h/cos(theta) - v*t
        k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
        t_shower = t
        t_light  = k/c
        t_total  = t_shower + t_light
        tt       = t_total

        num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
        num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
        zp       = ((num1_z - num2_z) / den)
        zm       = ((num1_z + num2_z) / den)
        v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
        v_B      = num2_z
        vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
        vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
        kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
        km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
        alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
        alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
        phip     = 90+theta-rad2deg(alphap)
        phim     = 90+theta-rad2deg(alpham)   
        vtp      = vp * sin(alphap)
        vtm      = vm * sin(alpham)
        omegap   = (vtp / kp)
        omegam   = (vtm / km)
        bp       = abs(omegap / (kp**2))
        bm       = abs(omegam / (km**2))

        ZM.append(zm*cos(theta)) 
        ZP.append(zp*cos(theta))
        BM.append(bm)            
        BP.append(bp)
        PHIP.append(phip)
        PHIM.append(phim)
        T.append(tt)
        
        '''
        if (iii==0) or (iii==2000):
            
            print("z:",z,"\nk:",k,"\nt_shower:",t_shower,"\nt_light:",t_light,"\ntt:",tt,"\nzp:",zp,"\nzm:",zm,"\nvp:",vp,"\nvm:",vm
                  ,"\nkp:",kp,"\nkm:",km,"\alphap:",alphap,"\nalpham:",alpham,"\nphip:",phip,"\nphim:",phim,"\nvtp:",vtp,"\nvtm:",vtm,
                  "\nomegap:",omegap,"\nomegam:",omegam,"\nbp:",bp,"\nbm:",bm)
            #clevbp = bp
            #clevbm = bm
        '''
    ZP, ZM, T, BP, BM, PHIP, PHIM = array(ZP), array(ZM), array(T), array(BP), array(BM), array(PHIP), array(PHIM)
    
    if detector==1:
        entry_bright1 = entry_brightness(T,h,c,v,theta,L)
    entry_bright = entry_bright1
    
    #indicesP = np.argsort(ZP)
    #indicesM = np.argsort(ZM)    
    #ZP,ZM, BP,BM, PHIP,PHIM, T = ZP[indicesP],ZM[indicesM], BP[indicesP],BM[indicesM], PHIP[indicesP],PHIM[indicesM], T[indicesP]
    
    Tn  = T * 1e+9
    TT  = (Tn-Tn.min())
    
    if 'b' in plotme:
        if ptype==('y' or 'Y'):
            print('Entry Brightness:', entry_bright)
            BM2 = BM/entry_bright
            BP2 = BP/entry_bright
        else:
            pass
    
    #if detector==2:
    #cond1 = (ZP<=h*cos(theta))
    #ZM, ZP, TT, BM, BP, PHIM, PHIP = ZM[cond1], ZP[cond1], TT[cond1], BM[cond1], BP[cond1], PHIM[cond1], PHIP[cond1]
    #print(TT.shape, PHIM.shape, PHIP.shape)
    #cond2 = (ZM>=0)
    #ZM, ZP, TT, BM, BP, PHIM, PHIP = ZM[cond2], ZP[cond2], TT[cond2], BM[cond2], BP[cond2], PHIM[cond2], PHIP[cond2]
    #print(TT.shape, PHIM.shape, PHIP.shape)
    #if BP[-1]:
    #    print("BP[-1]: ",BP[-1]) 
    
    if detector==1:
        mylabel='Central Detector'
    else:
        mylabel='Surrounding Detectors'
            
    if plotme == 'x vs t':
        plt.plot(TT[Z1n<=h*cos(theta)], Z1n[Z1n<=h*cos(theta)], c=color, ls='--')
        plt.plot(TT[Z1n<=h*cos(theta)], Z2n[Z1n<=h*cos(theta)], c=color, ls='-' , label=r'Detector {}'.format(detector))
        plt.axhline(zc_act, c='k', ls=':')
        plt.ylim(0,h)
        if L>1e-10:
            plt.text(4, round(zc_act,2)+0.075, r'$z_c = {} \; m$'.format(round(zc_act,2)), fontsize=16)
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'image height $z_{\pm}$ (m)'         , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
    


        
    elif plotme == 'b vs phi':
        plt.plot(PHIM, BM, c=color, ls='-.')
        plt.plot(PHIP, BP, c=color, ls='-' , label=mylabel)
        plt.xlabel(r'angular locations $\phi_{pm}\;(in\;degrees)$', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        plt.xlim(0,91)
        '''try:
            phi_max = PHIP[argmax(Tn[ZM>0])]
            plt.xlim(0,phi_max)
        except:
            plt.xlim(0,90)
            pass
    '''
    
    elif plotme == 'phi vs t':
        if L!=0:
            plt.plot(TT, PHIM, c=color, ls='-.')
            plt.plot(TT, PHIP, c=color, ls='-' , label=mylabel)
            plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
            plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'       , fontsize=18).set_rotation(90)
            plt.tick_params(axis='both', direction='in', labelsize=18)
            plt.ylim(-0.1,90.01)
            
            try:
                t_max = max(max(Tn[ZM>0]) , max(Tn[ZP<h*cos(theta)]))
                plt.xlim(-0.1,t_max)
            except:
                plt.xlim(-0.1,30)
                pass
    
    break
plt.legend()
plt.show()

In [ ]:
BP

In [ ]:
BM

In [ ]:
plt.plot(BP)
plt.yscale('log')

In [ ]:
plt.figure(figsize=(12,9))
n=50000
plt.plot(ZM[:n],'r')
plt.plot(ZP[:n],'b')
plt.plot(T[:n]*1e+8,'k')
plt.xlim(0,15000)
plt.ylim(2,5)
plt.legend()

In [ ]:
import numpy as np
args = np.argsort(T)
T2   = T[args]
ZM2  = ZM[args]
ZP2  = ZP[args]
plt.plot(T2,ZM2,'r')
plt.plot(T2,ZP2,'b')

In [ ]:
args = np.argsort(T)
T2   = T[args]
BM2  = BM[args]
BP2  = BP[args]
plt.plot(T2,BM2,'r')
plt.plot(T2,BP2,'b')
plt.yscale('log')

In [ ]:
plt.plot(T,BM,'r')
plt.plot(T,BP,'b')
plt.yscale('log')

In [ ]:
PHIM2 = PHIM[args]
PHIP2 = PHIP[args]

In [ ]:
#plt.figure(figsize=(10,8))
plt.plot(PHIM2,BM,'r')
plt.plot(PHIP2,BP,'b')
plt.yscale('log')

In [ ]:
plt.plot(PHIM,BM)
plt.plot(PHIP,BP)
plt.yscale('log')

In [ ]:
plt.plot(T2,PHIM2)
plt.plot(T2,PHIP2)
plt.ylim(0,90)

In [ ]:
phis = np.concatenate((PHIM,PHIP))
bs   = np.concatenate((BM  ,  BP))

In [ ]:
args2 = np.argsort(phis)
phis2 = phis[args2]
bs2   = bs  [args2]

In [ ]:
plt.plot(phis2,bs2)
plt.yscale('log')

In [ ]:
max(BP), min(BP)

In [ ]:
max(BM), min(BM)

In [ ]:
max(phis2)

In [ ]:
def entry_brightness(h,c,v,theta,L):
    den      = (c**2-v**2)
    z        = h/cos(theta)
    k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
    t_shower = 1e-10#2.3578794230541195e-8
    t_light  = k/c
    t_total  = t_shower + t_light
    tt       = t_total
    #print("Entry time:", tt)
    #num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
    num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
    #zp       = ((num1_z - num2_z) / den)
    #zm       = ((num1_z + num2_z) / den)
    v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
    v_B      = num2_z
    vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
    vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
    kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
    km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
    alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
    alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
    vtp      = vp * sin(alphap)
    vtm      = vm * sin(alpham)
    omegap   = (vtp / kp)
    omegam   = (vtm / km)
    bp       = abs(omegap / (kp**2))
    bm       = abs(omegam / (km**2))
    return bm

entry_brightness(h,c,v,theta,L=3.25)

In [ ]:
    Tn  = T * 1e+9
    TT  = (Tn-Tn.min())
    
    if 'b' in plotme:
        if ptype==('y' or 'Y'):
            print('Entry Brightness:', entry_bright)
            BM = BM/entry_bright
            BP = BP/entry_bright
        else:
            pass
     
    #if detector==2:
    #cond1 = (ZP<=h*cos(theta))
    #ZM, ZP, TT, BM, BP, PHIM, PHIP = ZM[cond1], ZP[cond1], TT[cond1], BM[cond1], BP[cond1], PHIM[cond1], PHIP[cond1]
    #print(TT.shape, PHIM.shape, PHIP.shape)
    #cond2 = (ZM>=0)
    #ZM, ZP, TT, BM, BP, PHIM, PHIP = ZM[cond2], ZP[cond2], TT[cond2], BM[cond2], BP[cond2], PHIM[cond2], PHIP[cond2]
    #print(TT.shape, PHIM.shape, PHIP.shape)
    #if BP[-1]:
    #    print("BP[-1]: ",BP[-1]) 
    
    if detector==1:
        mylabel=None#'Central Detector'
    else:
        mylabel='Surrounding Detectors'
            
    if plotme == 'x vs t':
        plt.plot(TT[Z1n<=h*cos(theta)], Z1n[Z1n<=h*cos(theta)], c=color, ls='--')
        plt.plot(TT[Z1n<=h*cos(theta)], Z2n[Z1n<=h*cos(theta)], c=color, ls='-' , label=r'Detector {}'.format(detector))
        plt.axhline(zc_act, c='k', ls=':')
        plt.ylim(0,h)
        if L>1e-10:
            plt.text(4, round(zc_act,2)+0.075, r'$z_c = {} \; m$'.format(round(zc_act,2)), fontsize=16)
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'image height $z_{\pm}$ (m)'         , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
    
    elif plotme == 'b vs t':
        plt.plot(TT[condition], BM[condition], c=color, ls='--')
        plt.plot(TT[condition], BP[condition], c=color, ls='-' , label=r'Detector {}'.format(detector))
        plt.axhline(1, c='k', ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{ground}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')

        
    elif plotme == 'b vs phi':
        plt.plot(PHIM, BM, c=color, ls='-.')
        plt.plot(PHIP, BP, c=color, ls='-' , label=mylabel)
        plt.xlabel(r'angular locations $\phi_{pm}\;(in\;degrees)$', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        try:
            phi_max = PHIP[argmax(Tn[ZM>0])]
            plt.xlim(0,phi_max)
        except:
            plt.xlim(0,90)
            pass
    
    
    elif plotme == 'phi vs t':
        if L!=0:
            plt.plot(TT, PHIM, c=color, ls='-.')
            plt.plot(TT, PHIP, c=color, ls='-' , label=mylabel)
            plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
            plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'       , fontsize=18).set_rotation(90)
            plt.tick_params(axis='both', direction='in', labelsize=18)
            plt.ylim(-0.1,90.01)
            try:
                t_max = max(max(Tn[ZM>0]) , max(Tn[ZP<h*cos(theta)]))
                plt.xlim(-0.1,t_max)
            except:
                plt.xlim(-0.1,30)
                pass
    break
    
plt.legend()
#plt.show()

In [ ]:
clevbp

In [ ]:
clevbm

In [ ]:
max(BM)

In [ ]:
plt.plot(PHIM)

In [ ]:
plt.plot(ZP,'b')
plt.plot(ZM,'r')
plt.plot(Tn,'k')
#plt.xlim(0,15000)
#plt.ylim(0,5)

In [ ]:
max(ZP[ZP<h*cos(theta)])

In [ ]:
BP[argmax(ZP[ZP<h*cos(theta)])]

In [ ]:
argmax(ZP[ZP<h*cos(theta)])

In [ ]:
ZP

In [ ]:
BM[argmax(ZM[ZM>=0])]

In [ ]:
max(BM[BM>=0])

In [ ]:
import numpy as np
a = np.array([1,1,1,1,1])
b = np.array([1,-1,1,1,1])

In [ ]:
suma = 0
for i in a:
    